In [1]:
import pandas as pd
import openpyxl
from tqdm import tqdm
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
from operator import itemgetter

import unidecode
import string
import re
from fuzzywuzzy import fuzz
import os

## Dependientes

### Población

In [2]:
##########################################################################################
poblacion = pd.read_stata("CEDE//poblacion.dta")
poblacion.columns = ["dane", "dpto", "municipio", "date", "population"]


##########################################################################################
data_to_add = []
dane_codes = poblacion['dane'].unique()

for code in dane_codes:
    subset = poblacion[poblacion['dane'] == code].iloc[0]
    dpto = subset['dpto']
    municipio = subset['municipio']
    for year in range(1990, 2015):
        data_to_add.append([code, dpto, municipio, year, np.nan])

df_to_add = pd.DataFrame(data_to_add, columns=['dane', 'dpto', 'municipio', 'date', 'population'])
poblacion_extended = pd.concat([poblacion, df_to_add])
poblacion = poblacion_extended.sort_values(by=['dane', 'date'])

#########################################
crecimiento = pd.read_excel("CEDE//crecimiento_poblacion.xlsx")
crecimiento["g"] = crecimiento["growth"]/100

poblacion = poblacion.merge(crecimiento, "left", "date")

##########################################################################################
def fill_population(data):
    for i in reversed(data.index[:-1]):  # Comenzamos desde el penúltimo y retrocedemos
        if pd.isna(data.at[i, 'population']):
            next_year_population = data.at[i+1, 'population']
            growth_rate = data.at[i, 'growth']
            data.at[i, 'population'] = next_year_population / (growth_rate / 100 + 1)
    return data


poblacion = poblacion.groupby(['dane']).apply(fill_population)
poblacion.reset_index(drop=True, inplace=True)

##########################################################################################
poblacion = poblacion[["dane", "date", "population"]]
poblacion = poblacion.rename(columns = {"dane":"codmpio"})


### Deuda (BANREP)

In [3]:
deuda = pd.read_excel("DEUDA.xlsx")
deuda = deuda[['id', 'entidad', 'tipo_entidad', 'id_dpt', 'año',
       'saldo_deuda($) ']]
deuda.columns = ["codmpio", "entidad", "tipo", "id_dpto", "date", "deuda"]
deuda = deuda[["codmpio", "id_dpto", "date", "deuda"]]

### Panel municipal

In [4]:
#Millones de pesos corrientes

In [5]:
bg = pd.read_stata("CEDE//PANEL_BUEN_GOBIERNO(2022).dta")
bg = bg[['codmpio', 'ano', 'y_total', 'y_corr', 'y_corr_tribut', 'y_no_tribut', 'y_transf', 'g_total', 'g_corr', 'g_func', 'g_cap', 'y_cap', 'y_cap_regalias', 'g_int_deudap', 'deficit_total', 'finan',  'finan_credito']]
bg = bg.rename(columns= {"ano": "date"})

### Merge

In [ ]:
df = pd.merge(poblacion, deuda, "left", ["codmpio", "date"])
df = df.merge(bg, "left", ["codmpio", "date"])
for column in df.columns[4:]:
    df[f"{column}_pc"] = df[f"{column}"]/df["population"]

In [ ]:
df.to_csv("dependientes.csv", index = False)